In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest
import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import MetaTrader5 as mt5
import pandas as pd
from get_data import get_data
import pytz
from datetime import datetime

import random
random.seed(42)

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


['Forex_Indicator',
 'CryptoCross_grp',
 'Crypto_group',
 'Energies_group',
 'Forex_group',
 'Indices_group',
 'Stocks_group']

In [4]:

tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Forex_group' in symbol.path and symbol.spread < 15]
print(tickers)


# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    # mt5.TIMEFRAME_M4,
    # mt5.TIMEFRAME_M5,
    mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_M30,
]

symbols = get_data(tickers, intervals, date_from, date_to)

['AUDUSDm', 'EURGBPm', 'EURUSDm', 'GBPUSDm', 'TRYDKKm', 'USDCHFm', 'USDJPYm']
MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AUDUSDm
AUDUSDm
AUDUSDm
AUDUSDm
AUDUSDm
EURGBPm
EURGBPm
EURGBPm
EURGBPm
EURGBPm
EURUSDm
EURUSDm
EURUSDm
EURUSDm
EURUSDm
GBPUSDm
GBPUSDm
GBPUSDm
GBPUSDm
GBPUSDm
TRYDKKm
TRYDKKm
TRYDKKm
TRYDKKm
TRYDKKm
USDCHFm
USDCHFm
USDCHFm
USDCHFm
USDCHFm
USDJPYm
USDJPYm
USDJPYm
USDJPYm
USDJPYm


In [5]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



{15: Timestamp('2021-10-27 17:15:00'),
 30: Timestamp('2021-10-27 17:15:00'),
 16385: Timestamp('2021-10-27 17:15:00'),
 16386: Timestamp('2021-10-27 17:15:00'),
 16388: Timestamp('2021-10-27 17:15:00')}

In [6]:
def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high

In [7]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib as ta
import numpy as np

class MeanReversion(Strategy):
    # Parámetros ajustables de la estrategia
    lookback_period = 50   # Periodo de la SMA
    deviation_threshold = 0.01  # 5% de desviación para generar señal
    risk = 3  # Tamaño del riesgo (posición)
    
    def init(self):
        # Calcular la media móvil simple (SMA)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.lookback_period)
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=2)

    def next(self):
        # Precio actual y valor de la SMA
        actual_close = self.data.Close[-1]
        sma_value = self.sma[-1]
        cum_rsi = self.rsi[-1] + self.rsi[-2]

        # Desviación del precio con respecto a la SMA (en porcentaje)
        deviation = (actual_close - sma_value) / sma_value

        if self.position:
            if self.position.is_long and actual_close >= self.sma:
                self.position.close()

            if self.position.is_short and actual_close <= self.sma:
                self.position.close()  
        
        else:
            # Condiciones para comprar (precio por debajo de la SMA más del umbral de desviación)
            if deviation <= -self.deviation_threshold and cum_rsi <= 25:
                self.buy(size=self.risk / 100)

            # Condiciones para vender (precio por encima de la SMA más del umbral de desviación)
            elif deviation >= self.deviation_threshold and cum_rsi >= 75:
                self.sell(size=self.risk / 100)


In [8]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    MeanReversion
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:

    try:
        start_date = intervals_start_dates[interval]

        bt_train = Backtest(
            symbols[ticker][interval].loc[start_date:], 
            strategy,
            commission=7e-4,
            cash=15_000, 
            margin=1/30
        )
        
        stats = bt_train.run()
        
        equity_curve = stats._equity_curve['Equity'].values    
        x = np.arange(len(equity_curve)).reshape(-1, 1)
        reg = LinearRegression().fit(x, equity_curve)
        stability_ratio = reg.score(x, equity_curve)

        df_stats = pd.DataFrame({
            'strategy':[strategy.__name__],
            'ticker':[ticker],
            'interval':[interval],
            'stability_ratio':[stability_ratio],
            'return':[stats['Return [%]']],
            'final_eq':[stats['Equity Final [$]']],
            'drawdown':[stats['Max. Drawdown [%]']],
            'drawdown_duration':[stats['Max. Drawdown Duration']],
            'win_rate':[stats['Win Rate [%]']], 
            'sharpe_ratio':[stats['Sharpe Ratio']],
            'trades':[stats['# Trades']],
            'avg_trade_percent':[stats['Avg. Trade [%]']],
            'exposure':[stats['Exposure Time [%]']],
            'final_equity':[stats['Equity Final [$]']],
            'Duration':[stats['Duration']],

        })

        performance = pd.concat([performance, df_stats])

    except:
        print(f'hubo un problema con {ticker} {interval}')


performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)


In [9]:

filter_performance = performance[
    (performance['return']>0) & (performance['return/dd']>2)
].sort_values(by=['drawdown_duration'], ascending=[True]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

,strategy,ticker,interval,stability_ratio,return,final_eq,drawdown,drawdown_duration,win_rate,sharpe_ratio,trades,avg_trade_percent,exposure,final_equity,Duration,return/dd,custom_metric
0,MeanReversion,AUDUSDm,16385,0.821254,13.772442,17065.866374,5.123439,392 days 03:00:00,63.934426,0.747872,122,0.117247,26.342783,17065.866374,1037 days 11:00:00,2.688125,10.823254


In [10]:
for ticker, interval in zip(portfolio, intervals):
    bt_train = Backtest(
        symbols[ticker][interval], 
        MeanReversion,
        commission=7e-4,
        cash=15_000, 
        margin=1/30
    )

    stats = bt_train.run()

    bt_train.plot(filename=f'./{ticker}.html', resample=False)

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


In [11]:
# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2021, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [mt5.TIMEFRAME_H1]

tickers = ['AUDUSDm']

symbols = get_data(tickers, intervals, date_from, date_to)
symbols

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288
AUDUSDm


{'AUDUSDm': {16385:                         Open     High      Low    Close  Volume  spread  \
  Date                                                                      
  2021-10-27 17:00:00  0.75338  0.75339  0.75263  0.75273     248      12   
  2021-10-28 09:00:00  0.75108  0.75128  0.75056  0.75115     636      12   
  2021-10-28 10:00:00  0.75111  0.75248  0.75087  0.75243     707      12   
  2021-10-28 11:00:00  0.75243  0.75283  0.75150  0.75204    1239      12   
  2021-10-28 12:00:00  0.75202  0.75246  0.75054  0.75101    2482      12   
  ...                      ...      ...      ...      ...     ...     ...   
  2024-08-30 16:00:00  0.67644  0.67648  0.67512  0.67578    1982       8   
  2024-08-30 17:00:00  0.67577  0.67617  0.67559  0.67598    1323       8   
  2024-08-30 18:00:00  0.67599  0.67668  0.67592  0.67662    1005       8   
  2024-08-30 19:00:00  0.67661  0.67686  0.67632  0.67671    1159       8   
  2024-08-30 20:00:00  0.67670  0.67703  0.67642  0.67649 

In [12]:
def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

In [13]:
from utils import walk_forward

lookback_bars = 1920
validation_bars = 200
warmup_bars = 80

params = {
    'deviation_threshold': list(np.arange(0.01, 0.1, 0.01)),
    # 'lookback_period': [12, 26, 50],
    'maximize': optim_func
}

stats = walk_forward(
    MeanReversion,
    symbols['AUDUSDm'][16385], 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=0.0002, 
    margin=1/30, 
    cash=15_000
)

  0%|          | 0/9 [00:00<?, ?it/s]

equity final: 15118.430409137998


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


  0%|          | 0/9 [00:00<?, ?it/s]

equity final: 15324.679391224003


...

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


  0%|          | 0/9 [00:00<?, ?it/s]

equity final: 17004.50768281804


  0%|          | 0/9 [00:00<?, ?it/s]

equity final: 16975.50566164804


In [14]:
from utils import plot_full_equity_curve

plot_full_equity_curve(
    symbols['AUDUSDm'][16385], 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True
)

In [15]:

from utils import get_wfo_stats

wfo_stats = get_wfo_stats(stats, warmup_bars, symbols['AUDUSDm'][16385])

for k, v in wfo_stats.items():
    print(k, v)

Start 2021-10-27 17:00:00
End 2024-08-30 20:00:00
Duration 1038 days 03:00:00
Exposure Time [%] 1.2905770964833183
Equity Final [$] 16975.50566164804
Equity Peak [$] 17023.291161648038
Return [%] 13.170037744320265
Buy & Hold Return [%] -10.128465718119376
Return (Ann.) [%] 4.020272580607576
Volatility (Ann.) [%] 5.11223751785375
Sharpe Ratio 0.7864017598101332
Sortino Ratio 1.1754695888805438
Calmar Ratio 0.5633373820359228
Max. Drawdown [%] -7.136527254907454
Avg. Drawdown [%] -0.4922550987164236
Max. Drawdown Duration 303 days 04:00:00
Avg. Drawdown Duration 12 days 23:00:00
# Trades 117
Win Rate [%] 63.24786324786324
Best Trade [%] 1.6352676378689868
Worst Trade [%] -2.3500763868503416
Avg. Trade [%] 0.11999284051638082
Max. Trade Duration 6 days 22:00:00
Avg. Trade Duration 2 days 01:00:00
Profit Factor 1.4761161401593375
Expectancy [%] 0.12305356705181265
SQN 1.735717058336444


In [16]:
stats[9]._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


In [17]:
trades = pd.DataFrame()
for stat in stats:
    trades = pd.concat([trades, stat._trades])

trades.head(50)

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,18897.0,201.0,218.0,0.714373,0.72064,118.430409,0.008773,2022-02-24 15:00:00,2022-02-25 08:00:00,0 days 17:00:00
0,-18327.0,156.0,169.0,0.742421,0.73496,136.746654,0.010050,2022-03-07 02:00:00,2022-03-07 15:00:00,0 days 13:00:00
1,18847.0,184.0,212.0,0.728446,0.73231,72.831216,0.005305,2022-03-08 06:00:00,2022-03-09 10:00:00,1 days 04:00:00
2,19017.0,278.0,279.0,0.725385,0.72521,-3.328888,-0.000241,2022-03-14 03:00:00,2022-03-14 04:00:00,0 days 01:00:00
0,19013.0,78.0,129.0,0.725385,0.72246,-55.613938,-0.004032,2022-03-14 03:00:00,2022-03-16 06:00:00,2 days 03:00:00
1,-18838.0,143.0,205.0,0.729464,0.73821,-164.755679,-0.011990,2022-03-16 20:00:00,2022-03-21 10:00:00,4 days 14:00:00
0,-17894.0,264.0,279.0,0.759668,0.75779,33.605576,0.002472,2022-04-05 05:00:00,2022-04-05 20:00:00,0 days 15:00:00
0,-17934.0,64.0,94.0,0.759668,0.75704,47.131198,0.003459,2022-04-05 05:00:00,2022-04-06 11:00:00,1 days 06:00:00
1,18246.0,110.0,191.0,0.749000,0.74440,-83.927403,-0.006141,2022-04-07 03:00:00,2022-04-12 12:00:00,5 days 09:00:00
0,18525.0,177.0,224.0,0.733647,0.72238,-208.715617,-0.015357,2022-04-22 06:00:00,2022-04-26 05:00:00,3 days 23:00:00
